In [49]:
# Imports
from pathlib import Path
import os
import rasterio
from rasterio.windows import Window
import PIL
import torchvision
import numpy
import cv2
import skimage
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from albumentations import PadIfNeeded
import pandas as pd
from tqdm import tqdm
import shutil
from animaloc.data import ImageToPatches, PatchesBuffer, save_batch_images
import random

# Splitting data for annotation

### For GeoTiffs

In [ ]:
def save_patches(img_path:Path,dest_dir:Path,tilesize=512):

    # create directory
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
        os.mkdir(dest_dir)
        print("emptying directory:", dest_dir)
    else:
        os.mkdir(dest_dir)
        print("creating directory:", dest_dir)
    
    # window reading with rasterio
    handler = rasterio.open(img_path)
    height, width = handler.meta['height'], handler.meta['width']
    coordinates = dict()
    count = 0
    for i,j in tqdm(product(list(range(0,height,tilesize)),list(range(0,width,tilesize)))):
        window = Window(j, i, tilesize, tilesize)
        
        try:
            chunk = handler.read(window=window)
            c,h,w = chunk.shape
            xmin, xmax = j, j+w
            ymin, ymax = i, i+h
            x_center = 0.5*(xmin+xmax)
            y_center = 0.5*(ymin+ymax)
            n_unique = np.unique(chunk).size
            if n_unique == 1:
                continue
            count += 1
            filename = img_path.name.split('.')[0] + f"-{j}-{i}.png"
            coordinates[count] = [xmin,xmax,ymin,ymax,x_center,y_center,filename]

            # save to disk
            chunk = np.transpose(chunk,(1,2,0))
            skimage.io.imsave(dest_dir/filename,chunk)          
                
        except Exception as e:
            print("Failed for",(i,j),e)
            pass

    cols = ['xmin','xmax','ymin','ymax','x_center','y_center','filename']
    coordinates = pd.DataFrame.from_dict(coordinates,
                                        orient='index',
                                        columns=cols)
    coordinates.to_csv(dest_dir/f"coordinates{img_path.name.split('.')[0]}.csv",index=False)
    handler.close()
    return coordinates

In [ ]:
# for datapath in Path("../annotation_data/camp6/").iterdir():
datapath = Path("../annotation_data/camp6/150m_RGB.tif")
dest_dir = datapath.parent/(datapath.name.split('.')[0])
# coordinates =  save_patches(datapath,dest_dir,tilesize=1024)

In [ ]:
coordinates

### For Images (jpg, rgb etc.)
Using Herdnet code

In [ ]:
!pwd # current working dir

In [ ]:
# splitting validation data
Path("../general_dataset/val_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/val 640 640 100 \
    ../general_dataset/val_splits \
    -csv ../general_dataset/groundtruth/csv/val_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [ ]:
# splitting training data
Path("../general_dataset/train_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/train 640 640 100 \
    ../general_dataset/train_splits \
    -csv ../general_dataset/groundtruth/csv/train_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [ ]:
# splitting test data
Path("../general_dataset/test_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/test 640 640 100 \
    ../general_dataset/test_splits \
    -csv ../general_dataset/groundtruth/csv/test_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [41]:
# splitting savmap data
Path("../savmap_dataset_v2/images_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../savmap_dataset_v2/images 640 640 100 \
    ../savmap_dataset_v2/images_splits \
    -csv ../savmap_dataset_v2/gt.csv \
    -min 0.0 -all False

Exporting patches: 100%|██████████████████████| 654/654 [18:38<00:00,  1.71s/it]


### Sampling emtpy and non empty images

In [50]:
# sample training data

directory = Path(r"../savmap_dataset_v2/train/")
labels = directory/'labels'
dest_images = directory/'images_nonempty'
source_images = directory/'images'

# create destination images
dest_images.mkdir(exist_ok=True)

# move non empty images
# -- Uncomment to run > Be careful
# for file in labels.iterdir():
#     img_name = file.name.split('.')[0]+'.JPG'
#     if (source_images/img_name).exists():
#         os.rename(src=source_images/img_name,
#                 dst=dest_images/img_name)

# # and empty 
# -- Uncomment to run > Be careful!!
# num_non_empty = len(list(labels.iterdir()))
# num_empty_target = num_non_empty
# empty_images = list(source_images.iterdir())
# random.seed(41) # seeding for reproducibility
# random.shuffle(empty_images) # shuffle
# for file in empty_images[:num_empty_target]:
#     img_name = file.name
#     os.rename(src=file,
#                 dst=dest_images/img_name)

# rename folders
# os.rename(src=source_images,dst=directory/'images_empty')
# os.rename(src=dest_images,dst=directory/'images')


In [53]:
len(list((directory/'images_empty').iterdir())),\
    len(list((directory/"images").iterdir())),\
        len(list((directory/"labels").iterdir()))


(27857, 5500, 2750)

# Visualizing data

In [ ]:
import albumentations as A
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# patch_size = 640
# num_classes = 7
down_ratio = 1

val_dataset = CSVDataset(
    csv_file = '../wildlife_localizer_data/val/gt.csv',
    root_dir = '../wildlife_localizer_data/val',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='bbox')]
    )

In [ ]:
val_dataset.anno_type

In [ ]:
labels = pd.read_csv('../wildlife_localizer_data/val/gt.csv')
labels.head()

In [ ]:
np.sort(labels['labels'].unique())

In [ ]:
# loader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

In [ ]:
images, targets = val_dataset[0]

In [ ]:
targets

In [ ]:
# img = np.transpose(images.numpy(),(1,2,0))
# plt.imshow(img)

# Creating YOLO labels

In [ ]:
# Images dimensions
height,width = 640, 640

# Create label directory
for directory in Path("../wildlife_localizer_data/").iterdir():
    if not directory.is_dir():
        continue 
    labels_dir = Path(os.path.join(directory,'labels'))
    labels_dir.mkdir(exist_ok=True,parents=False) # create directory if it does not exist
    labels = pd.read_csv(os.path.join(directory,'gt.csv'))
    #-- Saving labels in YOLO format
    for img_filename,df_group in tqdm(labels.groupby(by='images'),desc=directory.name):
        df_group['width'] = (df_group['x_max'] - df_group['x_min'])/width 
        df_group['height'] = (df_group['y_max'] - df_group['y_min'])/height
        df_group['x'] = (0.5*(df_group['x_min'] + df_group['x_max']))/width # x center
        df_group['y'] = (0.5*(df_group['y_min'] + df_group['y_max']))/height # y center
        df_group['labels'] = 0 

        # print('\n\n',directory.name,'\n',df_group[['labels','x','y','width','height']])
        # break

        # uncomment to save labels files
        labels_filename     = img_filename.split('.')[0] + '.txt'
        if len(df_group)>0:
            cols = ['labels','x','y','width','height']
            df_group[cols].to_csv(os.path.join(labels_dir,labels_filename),
                                    sep=" ",
                                    header=False,
                                    index=False)


In [45]:
# for savmap data
directory = Path("../savmap_dataset_v2/train/")
labels_dir = Path(os.path.join(directory,'labels'))
labels_dir.mkdir(exist_ok=True,parents=False) # create directory if it does not exist
labels = pd.read_csv(os.path.join(directory,'gt.csv'))
#-- Saving labels in YOLO format
for img_filename,df_group in tqdm(labels.groupby(by='images'),desc=directory.name):
    df_group['width'] = (df_group['x_max'] - df_group['x_min'])/width 
    df_group['height'] = (df_group['y_max'] - df_group['y_min'])/height
    df_group['x'] = (0.5*(df_group['x_min'] + df_group['x_max']))/width # x center
    df_group['y'] = (0.5*(df_group['y_min'] + df_group['y_max']))/height # y center
    df_group['labels'] = 0 

    # print('\n\n',directory.name,'\n',df_group[['labels','x','y','width','height']])
    # break

    # uncomment to save labels files
    labels_filename = img_filename.split('.')[0] + '.txt'
    if len(df_group)>0:
        cols = ['labels','x','y','width','height']
        df_group[cols].to_csv(os.path.join(labels_dir,labels_filename),
                                    sep=" ",
                                    header=False,
                                    index=False)

train: 100%|██████████| 2750/2750 [00:09<00:00, 303.35it/s]


In [46]:
# Control splitting
num_missing = 0
num_found = 0
num_total = 0
for path in Path("../savmap_dataset_v2/train/images").iterdir():
    filename = path.name.split('.')[0]
    labelpath = Path("../savmap_dataset_v2/train/labels")/(filename + '.txt')
    num_total += 1
    if not labelpath.exists():
        num_missing += 1
    else:
        num_found += 1

num_missing,num_found,num_total

(30607, 2750, 33357)

In [ ]:
labels = pd.read_csv(os.path.join(directory,'gt.csv'))
labels['labels'].unique()

# Savmap data
Saving bounding boxes in VOC format

In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import torch 
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import nms
from torchvision.transforms import PILToTensor
from PIL import Image

In [3]:
annotations_path = Path("../savmap_dataset/savmap_annotations_2014.shp")

In [4]:
data = gpd.read_file(annotations_path)
data.head()

,IMAGEUUID,TAGUUID,geometry
0,f77f4af5a1344b9086b307d2b4ba61ff,a9b3a2325dbe4a208bc3ae37eeb8e1e1,"POLYGON ((1197.000 568.000, 1186.000 568.000, ..."
1,33c79ba79aca4b06ae30a109e5cd868f,4999d35b3b8b407f8cbf48d95e689899,"POLYGON ((2689.000 778.000, 2674.000 747.000, ..."
2,12fe8b054f634ed0a0ffc5d7ed614b66,8d61ca1969b149cbaed4a9bd112c771f,"POLYGON ((3157.000 2721.000, 3224.000 2725.000..."
3,18c3f7ef05c74a849d2262786d71819e,5be6cc29725140b58e8d5fe5eba9b379,"POLYGON ((3559.000 2707.000, 3571.000 2708.000..."
4,18c3f7ef05c74a849d2262786d71819e,0093c3c4285a4eeaba82f29fb86b3c8a,"POLYGON ((437.000 606.000, 420.000 621.000, 42..."


In [6]:
# data.loc[data['IMAGEUUID']=='0a3ed15cfab4453795564140e8fde8ba']

In [7]:
# uuid = '0a3ed15cfab4453795564140e8fde8ba'
# polygons = data.loc[data['IMAGEUUID']==uuid,'geometry']
# polygons

In [9]:
bboxes = dict()
count = 0
pil_to_tensor = PILToTensor()
for uuid in tqdm(np.unique(data.IMAGEUUID),desc='Getting bbox'):

    # identifier, filenmae w/o suffix
    uuid = str(uuid)

    # load img as tensor
    path_to_img = f"../savmap_dataset/{uuid}.JPG"
    img_pil = Image.open(path_to_img)
    img_tensor = pil_to_tensor(img_pil)

    # get boxes
    polygons = data.loc[data['IMAGEUUID']==uuid,'geometry']
    boxes = np.array([list(polygon.bounds) for polygon in polygons])
    boxes = torch.from_numpy(boxes).float()

    # apply non max suppression o discard overlaping polygons
    areas = abs((boxes[:,2] - boxes[:,0])*(boxes[:,3] - boxes[:,1]))
    indices = nms(boxes=boxes,
                  scores= 1/areas, # discarding larger bbox when they overlap
                  iou_threshold=0.1)
    bbox = boxes[indices].numpy()

    # save bbox
    for idx,i in enumerate(range(count,count+bbox.shape[0])):
        bboxes[i] = [uuid,] + bbox[idx].tolist()
    count = count+bbox.shape[0]
# print('retained bbox indes:',indices)

Getting bbox:   0%|          | 0/654 [00:00<?, ?it/s]

Getting bbox: 100%|██████████| 654/654 [02:37<00:00,  4.17it/s]


In [38]:
columns=['images','x_min','y_min','x_max','y_max']
gt_bboxes = pd.DataFrame.from_dict(data=bboxes,
                       orient='index',
                       columns=columns)

for col in columns:
    if col != 'images':
        gt_bboxes[col] = gt_bboxes[col].apply(int)
    else:
        gt_bboxes[col] = gt_bboxes[col].apply(lambda x: f"{x}.JPG")

gt_bboxes['labels'] = 0 # class
gt_bboxes.head()

,images,x_min,y_min,x_max,y_max,labels
0,003a34ee6b7841e6851b8fe511ebe102.JPG,1679,858,1695,875,0
1,003a34ee6b7841e6851b8fe511ebe102.JPG,1503,962,1537,996,0
2,0078d29a8d0b489caa3425969c7477ac.JPG,2781,541,2822,577,0
3,0078d29a8d0b489caa3425969c7477ac.JPG,543,1725,603,1754,0
4,0078d29a8d0b489caa3425969c7477ac.JPG,620,1653,677,1689,0


In [36]:
# discarding invalid coordinates
# gt_bboxes.loc[gt_bboxes.min(axis=1,numeric_only=True)>=0]

In [40]:
# save valid bounding boxes
# mask = gt_bboxes.min(axis=1,numeric_only=True)>=0
# gt_bboxes.loc[mask].to_csv("../savmap_dataset_v2/gt.csv",sep=',',index=False)

In [ ]:
# Example: drawing bounding boxes

# load img as tensor
filename = gt_bboxes['filename'].sample(1).iloc[0]
path_to_img = f"../savmap_dataset/{filename}"
img_pil = Image.open(path_to_img)
img_tensor = PILToTensor()(img_pil)

boxes = gt_bboxes.loc[gt_bboxes['filename'] == filename, ['xmin','ymin','xmax','ymax']].to_numpy()
boxes = torch.from_numpy(boxes)

img_with_box = draw_bounding_boxes(img_tensor,
                                   boxes=boxes,
                                   colors="red",
                                   width=5).numpy().transpose((1,2,0))

plt.figure(figsize=(15,7))
plt.imshow(img_with_box)
plt.title(filename)
plt.show()